In [ ]:
from bokeh.io import output_notebook

output_notebook()

## Data preparation

In [ ]:
import pandas as pd

file = "../data/csv_files/US_census.csv"
df = pd.read_csv(file)

new_df = df.groupby("state", as_index=False).agg(
    income=("median_household_income", "mean"), density=("density", "mean")
)

bins = [-1, 39_000, 50_000, 60_000, 70_000, 100_000]
labels = ["<$40k", "$40k-$50k", "$50k-$60k", "$60k-$70k", ">$70k"]
colors = ["#f1e0e3", "#d8bbc4", "#ba93ab", "#966B93", "#6B467A"]

new_df["income_range"] = pd.cut(new_df["income"], bins=bins, labels=labels, right=False)
new_df["colors"] = pd.cut(new_df["income"], bins=bins, labels=colors, right=False)

new_df.head()

In [ ]:
from bokeh.sampledata.us_states import data

us_df = (
    pd.DataFrame(data)
    .T.loc[lambda df: ~df["name"].isin(["Alaska", "Hawaii"])]
    .reset_index(drop=True)
    .rename(columns={"name": "state"})
    .drop(columns="region")
)

us_df.head()

In [ ]:
plot_df = new_df.merge(us_df, on="state")
plot_df.head()

## Plotting

In [ ]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show

source = ColumnDataSource(data=plot_df)

p = figure(
    width=900,
    height=600,
    tooltips=[("State", "@state"), ("Average income", "@income_range")],
)

p.patches(
    "lons",
    "lats",
    fill_color="colors",
    line_color="black",
    legend_field="income_range",
    source=source,
)

p.yaxis.visible = False
p.xaxis.visible = False
p.grid.grid_line_color = None
p.legend.location = "bottom_left"

show(p)